# Data Anonymization Pipeline

In [ ]:
import pandas as pd
import uuid

output_path = "data/01_anonymized/"

## Users
We fetch the users enrolled in the course (specified by the CourseID 443) from the database. Anonymization will take place later since we need the information to map the fetched data to the users and can not anonymize it yet. We will use the `uuid` library to generate unique identifiers for each user.

In [ ]:
%%sql
select distinct u.id, u.login, u.first_name, u.last_name, u.email
from jhi_user u
left join user_groups ug on u.id = ug.user_id
left join course c on  ug.user_groups = c.student_group_name
where c.id = 443

In [ ]:
# generate uuid for user
jhi_user["uuid"] = [str(uuid.uuid4()) for _ in range(len(jhi_user))]

def find_uuid_by_id(user_id):
    values = jhi_user.loc[jhi_user["id"] == user_id, "uuid"].values
    if len(values) == 0:
        return "NOT_FOUND"
    return values[0]

def find_uuid_by_login(login):
    values = jhi_user.loc[jhi_user["login"] == login, "uuid"].values
    if len(values) == 0:
        return "NOT_FOUND"
    return values[0]

def find_uuid_by_email_or_first_name_and_last_name(email, first_name, last_name):
    values = jhi_user.loc[(jhi_user["email"] == email) | ((jhi_user["first_name"] == first_name) & (jhi_user["last_name"] == last_name)), "uuid"].values
    if len(values) == 0:
        return "NOT_FOUND"
    return values[0]

def find_uuid_by_first_name_and_last_name_and_email(first_name, last_name, email):
    values = jhi_user.loc[(jhi_user["first_name"] == first_name) & (jhi_user["last_name"] == last_name) & (jhi_user["email"] == email), "uuid"].values
    if len(values) == 0:
        return "NOT_FOUND"
    return values[0]

## Competency
We fetch the competencies for the course with ID 443.

In [ ]:
%%sql
select c.id, c.taxonomy, c.title, c.mastery_threshold, c.soft_due_date
from competency c
where c.course_id = 443

In [ ]:
competency.to_csv(output_path + "competency.csv", index=False)

## Learning Path
We fetch the learning path for the course with ID 443. We anonymize the user's id.

In [ ]:
%%sql
select lp.user_id, lp.progress, lp.started_by_student
from learning_path lp
where lp.course_id = 443

In [ ]:
learning_path["user_id"] = learning_path["user_id"].apply(find_uuid_by_id)
learning_path.to_csv(output_path + "learning_path.csv", index=False)

## Competency Participation
We fetch the competency participation for the course with ID 443. We anonymize the user's id.

In [ ]:
%%sql
select cu.user_id, cu.competency_id, cu.progress, cu.confidence, cu.confidence_reason, cu.last_modified_date
from competency_user cu
join competency c on cu.competency_id = c.id
where c.course_id = 443

In [ ]:
competency_user["user_id"] = competency_user["user_id"].apply(find_uuid_by_id)
competency_user.to_csv(output_path + "competency_user.csv", index=False)

## Science Events
We fetch the science events for users enrolled in the course with ID 443. We anonymize the user's id.

In [ ]:
%%sql
select se.*
from science_event se
join jhi_user u on se.identity = u.login
where u.id in (
    select distinct u.id
    from jhi_user u
    left join user_groups ug on u.id = ug.user_id
    left join course c on  ug.user_groups = c.student_group_name
    where c.id = 443
)

In [ ]:
science_event["identity"] = science_event["identity"].apply(find_uuid_by_login)
science_event.to_csv(output_path + "science_event.csv", index=False)

## Participations
We fetch the student participation records for exercises in the course with ID 443. We anonymize the student's id.

In [ ]:
%%sql
select p.student_id, p.exercise_id, p.test_run
from participation p
join exercise e on p.exercise_id = e.id
where e.course_id = 443

In [ ]:
participation["student_id"] = participation["student_id"].apply(find_uuid_by_id)
participation.to_csv(output_path + "participation.csv", index=False)

## Participant Score
We fetch the participant score for the course with ID 443. We anonymize the user's id.

In [ ]:
%%sql
select ps.user_id, ps.exercise_id, ps.last_rated_score, ps.last_rated_points
from participant_score ps
join exercise e on ps.exercise_id = e.id
where e.course_id = 443

In [ ]:
participant_score["user_id"] = participant_score["user_id"].apply(find_uuid_by_id)
participant_score.to_csv(output_path + "participant_score.csv", index=False)

## Lecture
We fetch the lecture for the course with ID 443. We are only interested in the IDs, not the exact content.

In [ ]:
%%sql
select l.id
from lecture l
where l.course_id = 443

In [ ]:
lecture.to_csv(output_path + "lecture.csv", index=False)

## Lecture Units
We fetch the lecture units for the course with ID 443. We are only interested in the IDs, not the exact content.

In [ ]:
%%sql
select lu.id
from lecture_unit lu
join lecture l on lu.lecture_id = l.id
where l.course_id = 443

In [ ]:
lecture_unit.to_csv(output_path + "lecture_unit.csv", index=False)

## Exercise
We fetch the exercises for the course with ID 443. We are only interested in the IDs, not the exact content.

In [ ]:
%%sql
select e.id
from exercise e
where e.course_id = 443

In [ ]:
exercise.to_csv(output_path + "exercise.csv", index=False)

## Links between Competencies and Resources
We fetch the links between competencies and resources for the course with ID 443.

In [ ]:
%%sql
select ce.*
from competency_exercise ce
join competency c on ce.competency_id = c.id
where c.course_id = 443

In [ ]:
competency_exercise.to_csv(output_path + "competency_exercise.csv", index=False)

In [ ]:
%%sql
select cl.*
from competency_lecture_unit cl
join competency c on cl.competency_id = c.id
where c.course_id = 443

In [ ]:
competency_lecture_unit.to_csv(output_path + "competency_lecture_unit.csv", index=False)

## Exam Exercise Results
We fetch the participant score for the course with ID 469 (Digital exam was conducted in another course instance due to administrative preferences.). We anonymize the user's id.

In [ ]:
%%sql
select ps.user_id, ps.exercise_id, ps.last_rated_score, ps.last_rated_points
from participant_score ps
join exercise e on ps.exercise_id = e.id
join exercise_group eg on e.exercise_group_id = eg.id
join exam ex on eg.exam_id = ex.id
where ex.course_id = 469

In [ ]:
participant_score_exam["user_id"] = participant_score_exam["user_id"].apply(find_uuid_by_id)
participant_score_exam.to_csv(output_path + "participant_score_exam.csv", index=False)

## Lime Survey Responses
We map the users to the corresponding uuid and anonymize the user.

In [ ]:
lime_survey_pre = pd.read_csv("data/00_in/results-pre.csv")
lime_survey_pre["uuid"] = lime_survey_pre.apply(
    lambda row: find_uuid_by_first_name_and_last_name_and_email(
        row["firstname"], row["lastname"], row["email"]
    ),
    axis=1
)
lime_survey_pre = lime_survey_pre.drop(columns=["id", "submitdate", "lastpage", "startlanguage", "seed", "token", "firstname", "lastname", "email"])
lime_survey_pre.to_csv(output_path + "lime_survey_pre.csv", index=False)

In [ ]:
lime_survey_post = pd.read_csv("data/00_in/results-post.csv")
lime_survey_post["uuid"] = lime_survey_post.apply(
    lambda row: find_uuid_by_first_name_and_last_name_and_email(
        row["firstname"], row["lastname"], row["email"]
    ),
    axis=1
)
lime_survey_post = lime_survey_post.drop(columns=["id", "submitdate", "lastpage", "startlanguage", "seed", "token", "firstname", "lastname", "email"])
lime_survey_post.to_csv(output_path + "lime_survey_post.csv", index=False)

## Tutorial Group Mapping
We map each student to their assigned tutorial group by matching their information with the tutorial mapping data. We anonymize the student's identity by replacing personal information with their corresponding UUID.

In [ ]:
tutorial_mapping = pd.read_csv("data/00_in/tutorial_mapping.csv", delimiter=";")
tutorial_mapping["uuid"] = tutorial_mapping.apply(
    lambda row: find_uuid_by_email_or_first_name_and_last_name(
        row["E-MAIL"], row["VORNAME"], row["FAMILIENNAME"]
    ),
    axis=1
)
tutorial_mapping = tutorial_mapping[["uuid", "GRUPPE"]]
tutorial_mapping.rename(columns={"GRUPPE": "group"}, inplace=True)
tutorial_mapping.to_csv(output_path + "tutorial_mapping.csv", index=False)

## Finalize User
We export the anonymized user UUIDs to the output file, retaining only the UUID column and discarding all personal information.

In [ ]:
jhi_user[["uuid"]].to_csv(output_path + "jhi_user.csv", index=False)